# Introduction (2)
Attempt to plot multiple GPX files on one map

In [1]:
# Imports and setup
import gpxpy
import gpxpy.gpx
import pandas as pd
import plotly.express as px
from typing import TextIO

# Function to read GPX file

In [2]:
def read_gpx(file: TextIO) -> pd.DataFrame:
    with open(file, 'r', encoding='UTF-8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    route_info = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation
                })

    # Convert to pandas dataframe, remove first and last 100 points to protect personal information
    route_df = pd.DataFrame(route_info[100:-100])
    route_df['title'] = gpx.tracks[0].name
    route_df['start_time'] = gpx.time

    return route_df

In [3]:
read_gpx('Halve_Marathon_.gpx')

,latitude,longitude,elevation,title,start_time
0,51.471802,5.437449,19.1,Halve Marathon 💪😎,2021-11-20 10:20:55+00:00
1,51.471778,5.437420,19.1,Halve Marathon 💪😎,2021-11-20 10:20:55+00:00
2,51.471647,5.437231,19.1,Halve Marathon 💪😎,2021-11-20 10:20:55+00:00
3,51.471516,5.436992,19.2,Halve Marathon 💪😎,2021-11-20 10:20:55+00:00
4,51.471432,5.436863,19.2,Halve Marathon 💪😎,2021-11-20 10:20:55+00:00
...,...,...,...,...,...
1363,51.475500,5.447009,18.9,Halve Marathon 💪😎,2021-11-20 10:20:55+00:00
1364,51.475600,5.447208,18.9,Halve Marathon 💪😎,2021-11-20 10:20:55+00:00
1365,51.475687,5.447472,18.9,Halve Marathon 💪😎,2021-11-20 10:20:55+00:00
1366,51.475791,5.447754,18.9,Halve Marathon 💪😎,2021-11-20 10:20:55+00:00


In [4]:
route_df = read_gpx('Halve_Marathon_.gpx')

# Map made with Plotly
fig = px.line_mapbox(route_df, lat='latitude', lon='longitude', hover_data=['title'], height=700)
fig.update_layout(mapbox_style='open-street-map', mapbox_zoom=12, margin={'r': 0, 't': 0, 'l': 0, 'b': 0})
fig.update_traces(line=dict(width=5), hovertemplate='%{customdata[0]}')

# Read all .gpx files in the current folder

In [5]:
from pathlib import Path

df = None
for gpx_file in Path('.').glob('*.gpx'):
    route_df = read_gpx(gpx_file)

    if df is None:
        df = route_df
    else:
        df = df.append(route_df, ignore_index=True)

df.describe()

,latitude,longitude,elevation
count,4607.000000,4607.000000,4607.000000
mean,51.468235,5.431413,19.765661
std,0.012679,0.015405,1.531803
min,51.441785,5.391301,15.600000
25%,51.460051,5.426507,18.900000
50%,51.467008,5.434203,19.900000
75%,51.473006,5.442288,20.800000
max,51.502464,5.453902,28.200000


In [6]:
# Map made with Plotly
fig = px.line_mapbox(df, lat='latitude', lon='longitude', line_group='start_time', hover_data=['title', 'start_time'], height=700)
fig.update_layout(mapbox_style='open-street-map', mapbox_zoom=12, margin={'r': 0, 't': 0, 'l': 0, 'b': 0}, showlegend=False)
fig.update_traces(line=dict(width=5), hovertemplate='%{customdata[0]} (%{customdata[1]|%e %b %Y})<extra></extra>')
fig.show()

# First attempt at a heatmap

In [7]:
import numpy as np

def read_gpx_hm(file: TextIO, interpolate_num: int = 0) -> tuple[str, pd.DataFrame]:
    """Modified version of the read_gpx function to create heatmaps

    Args:
        file (TextIO): path of GPX file
        interpolate_num (int): interpolate n points between each existing succesive pair of points

    Returns:
        tuple[str, pd.DataFrame]: Title of activity and dataframe containing all coordinates
    """
    with open(file, 'r', encoding='UTF-8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    route_info = []
    prev = None
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                # Interpolate points
                if interpolate_num > 0:
                    if prev is not None:
                        for i in np.linspace(0, 1, interpolate_num+1, endpoint=False)[1:]:
                            route_info.append({
                                'latitude': i * point.latitude + (1-i) * prev[0],
                                'longitude': i * point.longitude + (1-i) * prev[1],
                                'elevation': i * point.elevation + (1-i) * prev[2]
                            })
                    prev = [point.latitude, point.longitude, point.elevation]
                # Add next point
                route_info.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation
                })

    # Convert to pandas dataframe
    route_df = pd.DataFrame(route_info)

    return gpx.tracks[0].name, route_df

def gpx_center(folder: Path) -> tuple[float, float]:
    """Computes the center of all gpx files in a certain folder.

    Args:
        folder (Path): folder to look for gpx files

    Returns:
        tuple[float, float]: latitude and longitude of center (average of all coordinates)
    """
    df = None
    for gpx_file in folder.glob('*.gpx'):
        route_df = read_gpx(gpx_file)

        if df is None:
            df = route_df
        else:
            df = df.append(route_df, ignore_index=True)
    
    return df.latitude.mean(), df.longitude.mean()

In [8]:
import plotly.graph_objects as go
hm = go.Figure()

df = None
for gpx_file in Path('.').glob('*.gpx'):
    title, route_df = read_gpx_hm(gpx_file, interpolate_num=3)
    # Line
    hm.add_trace( go.Scattermapbox(
        mode='lines',
        lat=route_df.latitude,
        lon=route_df.longitude,
        hovertext=title,
        hoverinfo='text',
        line={'color': 'rgba(0, 0, 255, 0.15)', 'width': .5}
    ) )

    if df is None:
        df = route_df
    else:
        df = df.append(route_df)
    # Heatmap
    #hm.add_trace( go.Densitymapbox(
    #    lat=route_df.latitude,
    #    lon=route_df.longitude,
    #    radius=8,
    #    hoverinfo='skip',
    #    below=''
    #) )
    
center_lat, center_lon = df.latitude.mean(), df.longitude.mean()
# Heatmap
hm.add_trace( go.Densitymapbox(
    lat=df.latitude,
    lon=df.longitude,
    radius=2,
    hoverinfo='skip',
    below=''
) )
hm.update_layout(
    margin = {'r': 0, 't': 0, 'l': 0, 'b': 0},
    mapbox = {
        'center': {'lat': center_lat, 'lon': center_lon},
        'style': 'open-street-map',
        'zoom': 12
    },
    showlegend=False,
    height=700
)
hm.show()

# Second attempt at a heatmap
Inspired by https://github.com/OGladfelter/strava_heatmap/

In [9]:
from pathlib import Path

df = None
for gpx_file in Path('.').glob('*.gpx'):
    route_df = read_gpx(gpx_file)

    if df is None:
        df = route_df
    else:
        df = df.append(route_df, ignore_index=True)

In [10]:
# Map made with Plotly
fig = px.line_mapbox(df, lat='latitude', lon='longitude', line_group='start_time', hover_data=['title', 'start_time'], height=700)
fig.update_layout(mapbox_style='carto-darkmatter', mapbox_zoom=12, margin={'r': 0, 't': 0, 'l': 0, 'b': 0}, showlegend=False)
fig.update_traces(line=dict(width=3, color="rgba(255,50,10,.3)"), hovertemplate='%{customdata[0]} (%{customdata[1]|%e %b %Y})<extra></extra>')
fig.show()